In [1]:
# Check if python is 3.9.5
import sys
print(sys.version)
%load_ext autoreload
%autoreload 2

3.9.5 (default, May 18 2021, 12:31:01) 
[Clang 10.0.0 ]


## Code questions

<div class="alert alert-block alert-warning">
    <b>When do I use <span style="font-family: Monospace; background-color: #FFFFE0">abc.ABC</span> as superclass and when <span style="font-family: Monospace; background-color: #FFFFE0"> abc.ABCMeta</span> as metaclass? </b> <br>
    <a href="https://docs.python.org/3/library/abc.html">Here</a> is the <span style="font-family: Monospace; background-color: #FFFFE0">abc</span> package.
</div>

**Answer:** For us it is not really important.

# Data mining

In [2]:
import multidb_wrapper
import database_classes
from naip_dbm import NAIPData

In [3]:
from shapely.geometry import Point

In [4]:
naip = NAIPData(directory="banasna")

In [5]:
lat = 44.545950
lon = -118.256980

pont = Point(float(lon),float(lat))

In [6]:
naip.run(pont)

logged in
https://naipeuwest.blob.core.windows.net/naip-index/rtree/tile_index.dat
https://naipeuwest.blob.core.windows.net/naip-index/rtree/tile_index.idx
https://naipeuwest.blob.core.windows.net/naip-index/rtree/tiles.p
A size of 100 meters was chosen for each tile dimension. For different resolutions and years, this corresponds to the following sizes in pixels:
    - 2013 (100 cm): 100 px
    - 2019 (60 cm): 166 px
    - 2018 (60 cm): 164 px
    - 2016 (60 cm): 164 px
    - 2013 (50 cm): 198 px
    - 2010 (100 cm): 100 px
    - 2015 (50 cm): 196 px
    - 2011 (100 cm): 100 px
    - 2017 (100 cm): 100 px
    - 2015 (100 cm): 100 px
    - 2017 (60 cm): 166 px
    - 2012 (100 cm): 100 px
    - 2016 (100 cm): 98 px
    - 2014 (100 cm): 100 px
v002/or/2016/or_100cm_2016/44118/m_4411830_se_11_1_20160721.tif
https://naipblobs.blob.core.windows.net/naip/v002/or/2016/or_100cm_2016/44118/m_4411830_se_11_1_20160721.tif
banasna/data_barrel/naip_western_europe_azure/v002/or/2016/or_100cm_2016/44

## NAIP

<div class="alert alert-block alert-danger">
    <b>What exactly is this <a href="https://planetarycomputer.microsoft.com/dataset/naip#Blob-Storage-Notebook">Azure blob storage</a>?</b> <br> It seems, as if it does not need authentication, but we would, if we used <a href="https://planetarycomputer.microsoft.com/dataset/naip#Example-Notebook">Planetary Computer STAC API</a> by Microsoft or even Google's <a href="https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ#description">Earth Engine Data Catalog</a>. Where is the caveat? 
</div>

**Answer:** The explanation was too long to reconstruct for me. But the blob storage is really a path to circumvent the access abrrier in this case.

<div class="alert alert-block alert-danger">
    <b>Are the NAIP tiles actually overlapping?</b> <br> I am unsure, whether a location query could lead to multiple tiles within a given year.
</div>

In [6]:
multidb_wrapper.set_directory("aa")

'aa/data_barrel'

Okay, how should I proceed?
- Implement the date in the query! This reduces the images retrieved to maybe one.
- Find the most centered image and start to stitch
- start off with fixed raster size
- use rasterio and fiona
- 

In [20]:
multidb_wrapper.set_locations([1,2,2,3], [2,3,4])

AssertionError: The lists for latitudes ({len(longitudes)}) and lon (4) have unequal length. Please adjust.

---

In [27]:
import numpy as np
import rasterio
from fiona.transform import transform
import geopandas as gpd
import matplotlib.pyplot as plt
from PIL import Image

# code is again taken from ref01 - I dropped tom augspurger a mail
IO_CRS = "epsg:4326"
point = pont
tile_size_pixels = 500
query_url = "https://naipblobs.blob.core.windows.net/naip/v002/or/2016/or_100cm_2016/44118/m_4411830_se_11_1_20160721.tif"
file_name = "aha.tif"

half_edge = int(tile_size_pixels/2)
with rasterio.open(query_url) as image:
    # here coordinates need to be shifted from the input crs to
    # the tif image crs and later converted to pixel
    point_in_img_crs = [p[0] for p in transform(
        IO_CRS, image.crs.to_string(), [point.x], [point.y])]
    point_in_img_pix = [
        int(np.floor(p)) for p in
        ~image.transform * point_in_img_crs]
    
    # a window around the tile can be produced
    wdw = rasterio.windows.Window(
        point_in_img_pix[0]-half_edge, point_in_img_pix[1]-half_edge,
        tile_size_pixels, tile_size_pixels)
    image_tile = image.read(window=wdw)
    kwargs = image.meta.copy()
    kwargs.update({
        'height': wdw.height,
        'width': wdw.width,
        'transform': rasterio.windows.transform(wdw, image.transform)})

    with rasterio.open(
        file_name, "w", **kwargs) as dst:
            dst.write(image_tile)  # PROBLEM: we loose the IR LAYER!!!!

            #dst.write_colormap(
            #           4, {
            #                0: (255, 0, 0, 255),
            #                255: (0, 0, 255, 255) })    

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 5696, 'height': 7646, 'count': 4, 'crs': CRS.from_epsg(26911), 'transform': Affine(1.0, 0.0, 395348.0,
       0.0, -1.0, 4935502.0), 'blockxsize': 512, 'blockysize': 512, 'tiled': True, 'compress': 'deflate', 'interleave': 'pixel'}


RasterioIOError: aha.tif: Currently, PHOTOMETRIC=YCBCR requires COMPRESS=JPEG

<div class="alert alert-block alert-danger">
    <b>How can I see if my image was stored properly in 4 layers?</b> <br> When saving as tif and switching the axes, do I loose a layer?
</div>